<a href="https://colab.research.google.com/github/asgarali429/Langchain-/blob/main/prompt_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install langchain-huggingface
!pip install streamlit
!pip install python-dotenv
!pip install pyngrok

In [ ]:
# Import necessary modules
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from google.colab import userdata

# Load Hugging Face API token securely from Colab userdata
hf_api_token = userdata.get('huggingfacehub_api_token')

# Configure the HuggingFace LLM using Mistral-7B
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    huggingfacehub_api_token=hf_api_token
)

# Wrap the LLM into a chat-friendly interface
model = ChatHuggingFace(llm=llm)

# Test basic interaction with the model
result = model.invoke("What is the capital of India?")
print(result.content)

In [ ]:
# Save the summary prompt template to a file
%%writefile template_generator.py

from langchain_core.prompts import PromptTemplate

# Define a detailed and structured prompt for summarizing research papers
template = PromptTemplate(
    template="""
Please summarize the research paper titled "{paper_input}" with the following specifications:
Explanation Style: {style_input}
Explanation Length: {length_input}
1. Mathematical Details:
   - Include relevant mathematical equations if present in the paper.
   - Explain the mathematical concepts using simple, intuitive code snippets where applicable.
2. Analogies:
   - Use relatable analogies to simplify complex ideas.
If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
Ensure the summary is clear, accurate, and aligned with the provided style and length.
""",
    input_variables=['paper_input', 'style_input', 'length_input'],
    validate_template=True
)

# Save prompt to file
template.save("template.json")

In [ ]:
# Save Hugging Face token in a .env file for secure use in Streamlit
with open(".env", "w") as f:
    f.write(f"HUGGINGFACEHUB_API_TOKEN={hf_api_token}")

In [ ]:
# Streamlit application: Research Paper Summary Generator
%%writefile app.py
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate

# Load API token from .env file
load_dotenv()
api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Configure the LLM
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    huggingfacehub_api_token=api_key
)
model = ChatHuggingFace(llm=llm)

# Streamlit UI elements
st.header("Research Tool")

# Dropdowns for user input
paper_input = st.selectbox("Select Research Paper Name", [
    "Attention Is All You Need",
    "BERT: Pre-training of Deep Bidirectional Transformers",
    "GPT-3: Language Models are Few-Shot Learners",
    "Diffusion Models Beat GANs on Image Synthesis"
])
style_input = st.selectbox("Select Explanation Style", [
    "Beginner-Friendly", "Technical", "Code-Oriented", "Mathematical"
])
length_input = st.selectbox("Select Explanation Length", [
    "Short (1-2 paragraphs)", "Medium (3-5 paragraphs)", "Long (detailed explanation)"
])

# Prompt template for generating the summary
template = PromptTemplate(
    template="""
Please summarize the research paper titled "{paper_input}" with the following specifications:
Explanation Style: {style_input}
Explanation Length: {length_input}
1. Mathematical Details:
   - Include relevant mathematical equations if present in the paper.
   - Explain the mathematical concepts using simple, intuitive code snippets where applicable.
2. Analogies:
   - Use relatable analogies to simplify complex ideas.
If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
Ensure the summary is clear, accurate, and aligned with the provided style and length.
""",
    input_variables=['paper_input', 'style_input', 'length_input'],
    validate_template=True
)

# Generate and display the summary
if st.button("Generate Summary"):
    chain = template | model
    result = chain.invoke({
        'paper_input': paper_input,
        'style_input': style_input,
        'length_input': length_input
    })
    st.write(result.content)

In [ ]:
# Kill existing Streamlit and ngrok processes if running
!pkill -f streamlit
!pkill -f ngrok

# Authenticate ngrok
!ngrok authtoken "Your ngrok token"  # Replace with your actual token

In [ ]:
# Launch Streamlit app and expose it using ngrok
from pyngrok import ngrok
import subprocess

# Run Streamlit app in the background
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# Create a public URL using ngrok
url = ngrok.connect(8501).public_url
print(f"Streamlit app running at: {url}")

In [ ]:
# Basic chatbot using LangChain prompt template and Mistral-7B
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from google.colab import userdata

# Retrieve API token
hf_api_token = userdata.get('huggingfacehub_api_token')

# Setup the LLM
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    huggingfacehub_api_token=hf_api_token
)
model = ChatHuggingFace(llm=llm)

# Create a chat template with memory
chat_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

chat_history = []

# Start chat loop
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    prompt = chat_template.invoke({'chat_history': chat_history, 'input': user_input})
    result = model.invoke(prompt)

    print(result.content)

    # Update chat history
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=result.content))